# Walkability analysis notebook

This notebook takes the data objects saved in `data/prepared/` and produced by `data_ingestion.ipynb`, and operates on them to create measures of walkability by geography.

This is achieved by computing the shortest walking distance to different categories of place, and averaging it across each geography.

This requires trips to be generated and analysed:

1. Constructing the starting points. We distribute starting points along the network nodes with a density proportional to the local population (at SA1 granularity)
2. Finding the end points. From each starting point, we want to obtain a sample of end points from point-of-interest data that is within some walking distance (e.g. 5km)
3. Constructing the trips. Each trip is a pair of a start point and the local end points, which are mapped to the geographically closest network node.
4. Compute the distance of all trips on the graph, using the edge weights, and take the minimum per category of destination, and the number within the search radius. Network libraries perform this efficiently.
5. This minimum distance and count per category are properties of the starting location. Statistics per geographic area can be computed by averaging them across the starting locations in the geography.
6. The walkability measures can be used in combination with other census data to investigate causal factors and correlates of walkability

